# Uncertainty quantification

In this notebook, we exemplify an uncertainty quantification (UQ) method as outlined in the paper "Uncertainty quantification, propagation and characterization by Bayesian analysis combined with global sensitivity analysis applied to
dynamical intracellular pathway models" by Eriksson and Jauhiainen et al. 

The purpose of inverse uncertainty quantification is to estimate unknown parameters of a model from observed data, and at the same time quantify the uncertainty in these parameter estimates. We do this in a Bayesian framework by characterizing
the posterior distribution of the parameters via Approxiamte Bayesian Computation (ABC) coupled to Markov chain Monte Carlo (MCMC). 

The contents of this Notebook are licensed as: CC BY 4.0 (https://creativecommons.org/licenses/by/4.0/)


In [ ]:
uqsaDirectory <- getwd()

## Prerequisites

In order to run this notebook, you need to have an R kernel installed, and the following packages installed within your R distribution (you may need to run `install.packages("[packageName]")` first):

In [ ]:
remotes::install_github("a-kramer/SBtabVFGEN")            # R package to generate vector field files from SBtab
remotes::install_github("a-kramer/rgsl")    # R package to solve ODEs with the C library "gsl"
remotes::install_github("icpm-kth/uqsa", subdir=c("UQ"))

# example of how to install an R package
# install.packages("VineCopula")

In [ ]:
library(SBtabVFGEN)
library(rgsl)
library(UQ)


library(parallel)
library(VineCopula)
library(MASS)
library(ks)
library(R.utils)
library(deSolve)
library(reshape2)
library(ggplot2)

The R package with the functions that we implemented for UQ are available at https://github.com/icpm-kth/uqsa, in the UQ subfolder.
To install the package, the following lines of code can be used:

## The model

We show our uncertainty quantification toolset applied to the small model (here denoted by "AKAR4") describing the interaction between the reporter protein AKAR4 and PKA, which is a small part of the model presented in "AKAP79 enables calcineurin to directly suppress protein kinase A activity" (Church TW, Tewatia P, Hannan S, et al. Elife. 2021;10:e68164. Published 2021 Oct 6. doi:10.7554/eLife.68164).

In particular, we work with ordinary differential equation (ODE) models. In general, an ODE model is described by the following:

\begin{cases}
&\dot{\mathbf{x}}(t) &&= f(\mathbf{x}(t), \mathbf{u}(t), \mathbf{p}) \nonumber \\
&\mathbf{x}(t_0) &&=\mathbf{x_0} \label{eq:model}\\
&\mathbf{y}(t) &&= g(\mathbf{x}(t),\mathbf{s}) \nonumber
\end{cases}

where
- $\mathbf{x}(t)$ corresponds to internal state variables (compounds, in our case), 
- $\mathbf{u}(t)$ to external input (species concentrations that are controlled by experimentalists),
- $\mathbf{y}(t)$ are the outputs (measurements from experiments),
- $\mathbf{p}$ are system parameters (on which we want to perform UQ),
- $\mathbf{s}$ are parameters for the readouts.

The parameters $\mathbf{\theta}=(\mathbf{p}, \mathbf{s})$ together with the initial conditions $\mathbf{x}(t_0)$ and the input $\mathbf{u}(t)$ fully specify the output from the system.



We use SBtab to describe the model and store experimental data.
Basing on the SBtab format, we created a folder with all the spreadsheet files in .tsv extension.
The information about the model is contained in the tables of compounds, parameters, reactions, input (not present in this simple model) and output.

## Import model from SBtab into R

To import the model from the SBtab spreadsheet files (see: https://www.sbtab.net), we implemented the R function `import_from_SBtab`. 
The input to this function is the path of the directry where all the SBtab files (in .tsv extension) are stored.
The output is a list (here called `model`) containing all the information about the model.

In [ ]:
SBtabDir <- paste0(uqsaDirectory,"/AKAR4") # write path of folder with all the SBtab (.tsv) files
setwd(SBtabDir)
model = import_from_SBtab(SBtabDir)

Here is some of the information that the list `model` contains.

In [ ]:
model$Compound
model$Parameter
model$Reaction
model$Output
model$Experiments
model$AKAR4_400nM[1:10,]

When running function `import_from_SBtab`, the command-line program VFGEN is called (see: https://warrenweckesser.github.io/vfgen/index.html), and it is used to generate the files `[modelName].R`, `[modelName]_gfv.c` and `[modelName]_gfv.h`, that are located in the same folder as the SBtab spreadsheets. These files generated thrugh VFGEN contain the functions that implement the ODE vector field of the model.

In our example, the model name is "AKAR4" and the vector field function is saved in file "AKAR.R":

In [ ]:
# load the R file with the vector field
modelName <- "AKAR4"
source(paste(SBtabDir,"/",modelName,".R",sep=""))

# print the vector field function
AKAR4

## Import experimental data from SBtab into R

Experimental data are also stored in the SBtab files. In particular, we created a table for each time series experiment, together with one table that summarized all the information about each experiment (i.e. initial conditions and input).

To import the experimental data we implemented the R function `import_experiments`, that requires as input the model name and the path of the folder with the .tsv files in the SBtab format. The output of `import_experiments` is a list, here denoted by `experiments`. Each element of this list corresponds to one experiment.

In this example, the experiments are time series data, that report the concentration of AKAR4 at several time points. 

In [ ]:
experiments <- suppressMessages(import_experiments(modelName, SBtabDir))

# print first experiment
names(experiments[[1]])
print(experiments[[1]][["initialState"]])

# Uncertainty Quantification

## Define priors for parameters

We decided to use a log-uniform distribution for the parameters, centered in their default value (read from the SBtab) and with lower and upper limit, respectively, 1000-fold larger and smaller than the default value:

\[defaulParameter / 1000 ; defaultParameter * 1000\]


In [ ]:
# default values of parameters
parVal <- model[["Parameter"]][["!DefaultValue"]]

# scale to determine the log uniform prior distribution
defRange <- 1000

# parIdx is the vector of indices of parameters on which we perform UQ.
# The other parameters are fixed at their default value.
# In this example we perform UQ on all the parameters.
parIdx <- 1:length(parVal)   


ll <- parVal/defRange
ul <- parVal*defRange
ll = log10(ll) # log10-scale
ul = log10(ul) # log10-scale

## Define the ABC setting

We will perform the UQ applying the ABCMCMC algorithm in multiple steps.
At each step we consider the data from one of the three experiments that we read from the SBtab tables.

In [ ]:
# Define the experiments to consider in each step of the ABCMCMC algorithm

experimentsIndices <- list(1,2,3)

npc <- 500    # number of samples for the Precalibration (npc) 
              # On a cluster, we can use 5000 or 50000.
p <- 0.01     # For the Pre-Calibration: Choose Top 1% samples with shortest distance to the experimental values


nChains <- 4  # For the ABC-MCMC Process: number of chains that are created in parallel 
ns <- 100     # number of samples in each ABC-MCMC chain (ns)
              # On a cluster, we can use 1000.
delta <- 0.01 # Distance threshold to compare experimental data and simulated data

# Define the number of Cores for parallelized computations
nCores <- 2

# Scaling factor, used to define the covariance matrix in the ABCMCMC 
sfactor <- 0.1  

# Choose whether to solve ODEs in R or C
environment <- "C"

set.seed(123456)

Create a customized distance function

In [ ]:
# Define the score function to compare simulated data with experimental data

getScore  <- function(yy_sim, yy_exp){
  yy_sim <- (yy_sim-0)/(0.2-0.0)
  ifelse(!is.na(yy_exp), yy_exp <- (yy_exp-108.6)/(183.9-108.6), Inf)
  distance <- mean((yy_sim-yy_exp)^2)
  return(distance)
}

## ABCMCMC

We start the UQ by running the ABCMCMC algorithm with data from the first experiment.

In [ ]:
#first set of experiments
expInd <- experimentsIndices[[1]]

The first step consists in creating a copula that will describe the prior distribution of parameters. 

In [ ]:
# Fit independent copula
out <- makeIndepCopula(ll, ul)
copula <- out$copula
U <- out$U
Z <- out$Z
Y <- out$Y

Using the copula, we can run functions `preCalibration` and `getMCMCPar`, that will determine a starting parameter (`startPar`) for each of the ABCMCMC chains, and a covariance matrix (`Sigma`) that will be used in the ABCMCMC algorithm.

In [ ]:
## Run Pre-Calibration Sampling
out1 <- preCalibration(experiments[expInd], modelName, parVal, parIdx, npc, copula, U, Z, getScore, nCores = nCores, environment)
  
## Get Starting Parameters from Pre-Calibration
out2 <- getMCMCPar(out1$prePar, out1$preDelta, p, sfactor, delta, nChains)
Sigma <- out2$Sigma
startPar <- out2$startPar
  

The ABCMCMC algorithm is performed by function `ABCMCMC`, that can be run in parallel for each chain.

In [ ]:
## Run ABC-MCMC Sampling
cl <- makeForkCluster(nChains)
draws <- parLapply(cl, 1:nChains, function(k) ABCMCMC(experiments[expInd], modelName, startPar[k,], parIdx, parVal, ns, Sigma, delta, U, Z, Y, copula, ll, ul, getScore, nCores, environment))
stopCluster(cl)

The output of `ABCMCMC` is a list, containing the draws of each chain. We now convert a list into a matrix with all the draws (from all the chains).

In [ ]:
# put draws together (from list to matrix)
draws <- do.call("rbind", draws)
pick <- !apply(draws, 1, function(rw) all(rw==0))
draws <- draws[pick,]

Let us inspect the matrix of draws `draws`.

In [ ]:
dim(draws)
head(draws)

Each row of matrix `draws` corresponds to a parameter that, when used in the ODE model, produces simulations that are "close enough" to the experimental data. This is illustrated by the following plot, produced by function `plotDataAndSimulations`: for each parameter draw produced by the ABCMCMC algorithm, we simulate the ODE system using the same initial conditions and input as the considered experiment (first experiment) and plot the simulated time series, together with the experimental data.

In [ ]:
source(paste0(SBtabDir,"/plotFunctionsAKAR4.R"))

plotDataAndSimulations(draws, expInd=1)

The empirical distribution of parameter draws is an approximation of the posterior distribution of parameters, gived data from the first experiment.

We fit a copula based on this posterior parameter distribution, and we use it as prior parameter distribution for the following UQ, where we will consider data from the second experiment.

In [ ]:
#Fitting Copula based on previous MCMC runs
out <- fitCopula(draws, ll, ul, nCores=1)        
copula <- out$copula
U <- out$U
Z <- out$Z
Y <- out$Y

#second set of experiments
expInd <- experimentsIndices[[2]]

## Run Pre-Calibration Sampling
out1 <- preCalibration(experiments[expInd], modelName, parVal, parIdx, npc, copula, U, Z, getScore, nCores = nCores, environment)
out2 <- getMCMCPar(out1$prePar, out1$preDelta, p, sfactor, delta, nChains)
Sigma <- out2$Sigma
startPar <- out2$startPar

## Run ABC-MCMC Sampling
cl <- makeForkCluster(nChains)
draws <- parLapply(cl, 1:nChains, function(k) ABCMCMC(experiments[expInd], modelName, startPar[k,], parIdx, parVal, ns, Sigma, delta, U, Z, Y, copula, ll, ul, getScore, nCores, environment))
stopCluster(cl)
                   
# put draws together (from list to matrix)
draws <- do.call("rbind", draws)
pick <- !apply(draws, 1, function(rw) all(rw==0))
draws <- draws[pick,]
               
dim(draws)
head(draws)


We use the draws, sampled through the ABCMCMC algorithm, to simulate trajectories.
We plot the trajectories (blue curves), together with the data from experiment 2 (black dots), using function `plotDataAndSimulations`.

In [ ]:
plotDataAndSimulations(draws, expInd=2)

The simulated time series agree with the experimental data from experiment 2.
This is reasonable, also due to the fact that we used data frmo experiment 2 in the ABCMCMC algorithm.

We now use the same draws applied to the same initial conditions as experiment 1, to see whether these parameters fit also the first experimental dataset.

In [ ]:
plotDataAndSimulations(draws, expInd=1)

Some of the draws lead to simulated time series (blue curves) that are "too far" from the first experiment data. Hence, we filter out such draws using function `checkFitWithPreviousExperiments`.

In [ ]:
# Checking fit with dataset 1
draws <- checkFitWithPreviousExperiments(2, experimentsIndices, modelName, 
                                         draws, experiments, parVal, parIdx, 
                                         getScore, delta, environment, nCores, nChains)

In [ ]:
dim(draws)
head(draws)

We plot again the simulated time series and the data point for the first and second experiment.

In [ ]:
plotDataAndSimulations(draws, expInd=1)

In [ ]:
plotDataAndSimulations(draws, expInd=2)

We now prceed with the UQ using the third dataset, using a prior distribution given by the copula, fit with the draws from the previous step. 

In [ ]:
#third set of experiments
expInd <- experimentsIndices[[3]]

#Fitting Copula based on previous MCMC runs
out <- fitCopula(draws, ll, ul, 1)        
copula <- out$copula
U <- out$U
Z <- out$Z
Y <- out$Y
  
## Run Pre-Calibration Sampling
out1 <- preCalibration(experiments[expInd], modelName, parVal, parIdx, npc, copula, U, Z, getScore, nCores = nCores, environment)
  
## Get Starting Parameters from Pre-Calibration
out2 <- getMCMCPar(out1$prePar, out1$preDelta, p, sfactor, delta, nChains)
Sigma <- out2$Sigma
startPar <- out2$startPar

## Run ABC-MCMC Sampling
cl <- makeForkCluster(nChains)
draws <- parLapply(cl, 1:nChains, function(k) ABCMCMC(experiments[expInd], modelName, startPar[k,], parIdx, parVal, ns, Sigma, delta, U, Z, Y, copula, ll, ul, getScore, nCores, environment))
stopCluster(cl)
                   
# put draws together (from list to matrix)
draws <- do.call("rbind", draws)
pick <- !apply(draws, 1, function(rw) all(rw==0))
draws <- draws[pick,]

In [ ]:
dim(draws)
head(draws)

We plot the timeseries data in the third experiments, and the simulated timeseries where the parameters in `draws` are used.

In [ ]:
plotDataAndSimulations(draws, expInd=3)

We compare the experimental datapoints of experiment 1 with the simulations obtained with the parameters in `draws` and the same initial conditions as in experiment 1.

In [ ]:
plotDataAndSimulations(draws, expInd=1)

We compare the experimental datapoints of experiment 2 with the simulations obtained with the parameters in `draws` and the same initial conditions as in experiment 2.

In [ ]:
plotDataAndSimulations(draws, expInd=2)

We filter out the parameters that produce simulations that are "too far" from the data in experiment 1 and 2.

In [ ]:
# Checking fit with dataset 1
draws <- checkFitWithPreviousExperiments(3, experimentsIndices, modelName, 
                                         draws, experiments, parVal, parIdx, 
                                         getScore, delta, environment, nCores, nChains)


In [ ]:
dim(draws)
head(draws)

We plot again simulations and experimental datapoint in the 3 experimental setups.

In [ ]:
plotDataAndSimulations(draws, expInd=1)

In [ ]:
plotDataAndSimulations(draws, expInd=2)

In [ ]:
plotDataAndSimulations(draws, expInd=3)

## Plot final results 

In [ ]:
parNames <- model[["Parameter"]][["!Name"]]
# histograms 
par(mfrow=c(2,3))
for(i in 1:3){
  hist(draws[,i], main=parNames[i], xlab = "Value in log scale")
}

# scatterplot for pairs of parameters
combinePar <- list(c(1,2), c(1,3), c(2,3))
for(i in combinePar){
  plot(draws[,i[1]], draws[,i[2]], xlab = parNames[i[1]], ylab = parNames[i[2]])
}


In [ ]:
# scatter plot
suppressMessages(library(plotly))
par(mfrow=c(1,1))
df = as.data.frame(draws)
colnames(df) <- parNames
plot_ly(dat = df, x = ~kf_C_AKAR4, y = ~kb_C_AKAR4, z = ~kcat_AKARp, type="scatter3d", mode="markers", marker=list(size = 1, color = "red"))



# Additional information about the methods

## ABC-MCMC with copulas

In ABC, samples from a prior distribution (or a proposal distribution) are accepted if the experimental data are reproduced by simulations from the model within a certain margin, so that a distance measure $\rho(S(\mathbf{y}^{sim}),S(\mathbf{y}^\text{exp}))$ is smaller than some predefined cut-off $\delta$ (S is a summary statistic of the data). The accepted parameter sets $\mathbf{\theta}$ will form the approximate posterior distribution $f(\mathbf{\theta}|\rho(S(\mathbf{y}^\text{exp}),S(\mathbf{y}^{sim}))\leq\delta)$. 

We use a distance measure, or scoring function, that accounts for deviations on both the model output (y-axis) and model input (x-axis) since the experimental data may be subject to experimental error both regarding input and output.  

We couple ABC to Markov chain Monte Carlo (MCMC) to efficiently sample from the posterior distribution of the parameters of interest. The model is fitted to the datasets in sequence. In order to incorporate the information from each fit, we do the following:

* Fit the model to one datataset, producing a posterior distribution
* Fit a copula to the posterior distribution 
* Use the copula as a prior to fit the model to the next dataset

We use graphical models called R-vines to formulate copulas that are constructed in pairs in order to describe the dependencies over multiple parameters. 



### Copulas

Copulas are multivariate probability distributions which describe the dependence structure between stochastic variables. The copula is a function that connects the multivariate distribution function to the marginal ones ($F_i$) as follows.

$$
F(\mathbf{x}) = C(F_1(x_1), \ldots, F_d(x_d))
$$ 

It can be shown that the multivariate (joint) density can be represented by a number of appropriate pair-copulas times the conditional marginal densities based on a pair-copula factorization. For a distribution over three variables, we have

$$
f(x_1, x_2, x_3) = c_{1,2}(F_1(x_1),F_2(x_2))c_{2,3|1} (F(x_2|x_1), F(x_3|x_1))c_{1,3}(F_1(x_1), F_3(x_3))f_1(x_1) f_3 (x_3) f_2(x_2)\\
$$

The copula pairs can be chosen independently of each other giving a wide range of possible dependence structures. We use graphical models called R-vines to arrange the pair copulas in a tree structure. 

Let's assume we work with a simple bivariate distribution and that we have a bivariate sample $\mathbf{x}=[\mathbf{x}_1, \mathbf{x}_2]$. For each variable $X_1$ and $X_2$, we estimate a cumulative distribution $P(X_1 \leq x_1)$ and $P(X_2 \leq x_2)$ (this can be done in R using e.g. the kcde or ecdf functions). We evaluate these cumulative densities in the sample points $[\mathbf{x}_1, \mathbf{x}_2]$ and denote this information $\mathbf{z}=[\mathbf{z}_1, \mathbf{z}_2]$. This step maps the sample points ($\mathbf{x}$) into the $[0,1]$ space and we fit a copula to $\mathbf{z}$ in this space.




### Pre-calibration

Each fit to a dataset begins with a pre-calibration step, in order to construct a covariance matrix for the MCMC transition kernel as well as finding starting points for the MCMC chains.

In the pre-calibration step we sample independently from the prior distribution a number of times ($npc$) to produce a set of samples. We generate simulated output from the model for each parameter set and check, using the distance measure, how close the simulated outputs $\mathbf{y}^{sim}$are to the experimental data $\mathbf{y}^{exp}$. We save a certain fraction $p$ of the samples that have the smallest distances. With these samples, we estimate a covariance matrix that is used as transition kernel Q (scaled appropriately) for the MCMC chains. We also pick random starting points for the chains. 

In order to perform a streamlined computational pipeline, we always formulate the prior distribution of the parameters in the form of a copula. For the fit to the first dataset, when we have uniform independent priors for all parameters, we use an independece copula, where all copula pairs (i.e. pairs of parameters) are independent. 